<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [8]:
## Task 1: Mark all launch sites on a map
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
from js import fetch
import io

print("="*70)
print("TASK 1: Mark all launch sites on a map")
print("="*70)

# First, load the data
print("\n📥 Loading data...")
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"
resp = await fetch(URL)
dataset_part_2_csv = io.BytesIO((await resp.arrayBuffer()).to_py())
df = pd.read_csv(dataset_part_2_csv)

print(f"✅ Data loaded successfully!")
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# Get unique launch sites with their coordinates
launch_sites_info = df[['LaunchSite', 'Latitude', 'Longitude']].drop_duplicates()

# Check if we have coordinates for all launch sites
print("\n📍 IDENTIFYING LAUNCH SITES:")
print("-" * 50)

if launch_sites_info['Latitude'].isna().any() or launch_sites_info['Longitude'].isna().any():
    print("⚠️  Warning: Some launch sites missing coordinates")
    # Show sites with missing coordinates
    missing_coords = launch_sites_info[launch_sites_info['Latitude'].isna() | launch_sites_info['Longitude'].isna()]
    print(f"Sites with missing coordinates: {list(missing_coords['LaunchSite'])}")

print(f"\nFound {len(launch_sites_info)} unique launch sites:")
print(launch_sites_info.to_string(index=False))

# Calculate center of all launch sites for map initialization
center_lat = launch_sites_info['Latitude'].mean()
center_lon = launch_sites_info['Longitude'].mean()

print(f"\n🌍 Map center coordinates: Latitude={center_lat:.4f}, Longitude={center_lon:.4f}")

# Create a base map
print("\n🗺️ Creating map...")
spacex_map = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=5,
    tiles='OpenStreetMap'
)

# Add title to the map
title_html = '''
             <h3 align="center" style="font-size:16px"><b>SpaceX Launch Sites</b></h3>
             '''
spacex_map.get_root().html.add_child(folium.Element(title_html))

# Create a MarkerCluster for grouping markers
marker_cluster = MarkerCluster().add_to(spacex_map)

# Add markers for each launch site
print("\n📍 Adding markers for launch sites...")
for idx, row in launch_sites_info.iterrows():
    launch_site = row['LaunchSite']
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Skip if coordinates are missing
    if pd.isna(latitude) or pd.isna(longitude):
        print(f"  ⚠️  Skipping {launch_site}: Missing coordinates")
        continue
    
    # Create popup with launch site information
    popup_content = f"""
    <div style="width: 200px;">
        <h4 style="margin: 5px 0;">{launch_site}</h4>
        <hr style="margin: 5px 0;">
        <p style="margin: 5px 0;"><b>Coordinates:</b></p>
        <p style="margin: 5px 0;">Latitude: {latitude:.4f}</p>
        <p style="margin: 5px 0;">Longitude: {longitude:.4f}</p>
    </div>
    """
    
    # Add marker to the cluster
    folium.Marker(
        location=[latitude, longitude],
        popup=folium.Popup(popup_content, max_width=250),
        tooltip=f"Click for {launch_site} info",
        icon=folium.Icon(
            color='red',
            icon='rocket',
            prefix='fa'
        )
    ).add_to(marker_cluster)
    
    print(f"  ✅ Added marker for {launch_site}")

# Add mouse position to show coordinates when hovering
print("\n🖱️ Adding mouse position display...")
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='Unable to get coordinates',
    lng_first=False,
    num_digits=4,
    prefix='Coordinates:',
    lat_formatter=lambda x: f'{x:.4f}°',
    lng_formatter=lambda x: f'{x:.4f}°'
).add_to(spacex_map)

# Add layer control
folium.LayerControl().add_to(spacex_map)

# Add launch site statistics to the map
print("\n📊 Adding launch statistics...")

# Calculate statistics for each launch site
if 'class' in df.columns:
    launch_stats = df.groupby('LaunchSite').agg({
        'FlightNumber': 'count',
        'class': 'mean'
    }).reset_index()
else:
    launch_stats = df.groupby('LaunchSite').agg({
        'FlightNumber': 'count'
    }).reset_index()
    launch_stats['class'] = None

launch_stats = launch_stats.rename(columns={
    'FlightNumber': 'TotalLaunches',
    'class': 'SuccessRate'
})

# Merge with coordinates
launch_sites_with_stats = pd.merge(launch_sites_info, launch_stats, on='LaunchSite')
if launch_sites_with_stats['SuccessRate'].notna().any():
    launch_sites_with_stats['SuccessRate'] = launch_sites_with_stats['SuccessRate'] * 100

# Add statistics as custom markers with DivIcon
for idx, row in launch_sites_with_stats.iterrows():
    launch_site = row['LaunchSite']
    latitude = row['Latitude']
    longitude = row['Longitude']
    total_launches = int(row['TotalLaunches'])
    success_rate = row['SuccessRate']
    
    # Create custom HTML for the marker label
    if not pd.isna(success_rate):
        label_text = f"""
        <div style="font-size: 12px; font-weight: bold; color: {'green' if success_rate >= 50 else 'red'};">
            {launch_site}<br>
            Launches: {total_launches}<br>
            Success: {success_rate:.1f}%
        </div>
        """
    else:
        label_text = f"""
        <div style="font-size: 12px; font-weight: bold; color: blue;">
            {launch_site}<br>
            Launches: {total_launches}
        </div>
        """
    
    # Add label marker slightly offset from the main marker
    folium.map.Marker(
        [latitude + 0.05, longitude],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=label_text
        )
    ).add_to(spacex_map)

# Add a legend
print("\n📖 Adding legend...")
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;">
     <b>Legend</b><br>
     <i class="fa fa-rocket" style="color:red"></i> Launch Site<br>
     <i class="fa fa-circle" style="color:green"></i> High Success<br>
     <i class="fa fa-circle" style="color:red"></i> Low Success<br>
     <i class="fa fa-circle" style="color:blue"></i> No Success Data
</div>
'''
spacex_map.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
output_file = 'spacex_launch_sites.html'
spacex_map.save(output_file)
print(f"\n💾 Map saved to: {output_file}")

# Display the map
print("\n🌐 Displaying map...")
display(spacex_map)

# Additional statistics
print("\n📈 LAUNCH SITE STATISTICS:")
print("-" * 50)

for idx, row in launch_sites_with_stats.iterrows():
    launch_site = row['LaunchSite']
    total_launches = int(row['TotalLaunches'])
    
    if not pd.isna(row['SuccessRate']):
        success_rate = row['SuccessRate']
        success_count = int((success_rate / 100) * total_launches)
        failure_count = total_launches - success_count
        
        print(f"\n🚀 {launch_site}:")
        print(f"   Total launches: {total_launches}")
        print(f"   Successful: {success_count} ({success_rate:.1f}%)")
        print(f"   Failed: {failure_count}")
    else:
        print(f"\n🚀 {launch_site}:")
        print(f"   Total launches: {total_launches}")
        print(f"   Success rate: Not available")

print("\n✅ TASK 1 COMPLETED!")
print("="*70)

TASK 1: Mark all launch sites on a map

📥 Loading data...
✅ Data loaded successfully!
Dataset shape: (90, 18)
Columns: ['FlightNumber', 'Date', 'BoosterVersion', 'PayloadMass', 'Orbit', 'LaunchSite', 'Outcome', 'Flights', 'GridFins', 'Reused', 'Legs', 'LandingPad', 'Block', 'ReusedCount', 'Serial', 'Longitude', 'Latitude', 'Class']

📍 IDENTIFYING LAUNCH SITES:
--------------------------------------------------

Found 3 unique launch sites:
  LaunchSite  Latitude   Longitude
CCAFS SLC 40 28.561857  -80.577366
 VAFB SLC 4E 34.632093 -120.610829
  KSC LC 39A 28.608058  -80.603956

🌍 Map center coordinates: Latitude=30.6007, Longitude=-93.9307

🗺️ Creating map...

📍 Adding markers for launch sites...
  ✅ Added marker for CCAFS SLC 40
  ✅ Added marker for VAFB SLC 4E
  ✅ Added marker for KSC LC 39A

🖱️ Adding mouse position display...

📊 Adding launch statistics...

📖 Adding legend...

💾 Map saved to: spacex_launch_sites.html

🌐 Displaying map...



📈 LAUNCH SITE STATISTICS:
--------------------------------------------------

🚀 CCAFS SLC 40:
   Total launches: 55
   Success rate: Not available

🚀 VAFB SLC 4E:
   Total launches: 13
   Success rate: Not available

🚀 KSC LC 39A:
   Total launches: 22
   Success rate: Not available

✅ TASK 1 COMPLETED!


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [9]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [10]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [11]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [12]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [13]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
from js import fetch
import io
import pandas as pd

# Try this corrected URL instead
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

try:
    resp = await fetch(URL)
    spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
    spacex_df = pd.read_csv(spacex_csv_file)
    
    print(f"✅ Data loaded successfully!")
    print(f"Dataset shape: {spacex_df.shape}")
    print(f"Columns available: {spacex_df.columns.tolist()}")
    
    # Select relevant sub-columns
    spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
    
    # Group by Launch Site to get unique sites with their coordinates
    launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
    launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
    
    print(f"\n📍 Launch sites found:")
    print(launch_sites_df.to_string(index=False))
    
except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("\n🔧 Troubleshooting tips:")
    print("1. Check your internet connection")
    print("2. The file might have been moved - check with your course instructor")
    print("3. Try accessing the URL directly in your browser")
    print("4. You might need to use a different dataset source")
    print("5. Verify the course materials for updated file locations")

✅ Data loaded successfully!
Dataset shape: (56, 13)
Columns available: ['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site', 'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome', 'class', 'Lat', 'Long']

📍 Launch sites found:
 Launch Site       Lat        Long
 CCAFS LC-40 28.562302  -80.577356
CCAFS SLC-40 28.563197  -80.576820
  KSC LC-39A 28.573255  -80.646895
 VAFB SLC-4E 34.632834 -120.610745


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [14]:
# Task 2:import folium
import pandas as pd
from folium.plugins import MarkerCluster

# Load the SpaceX data (use your corrected URL)
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

try:
    resp = await fetch(URL)
    spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
    spacex_df = pd.read_csv(spacex_csv_file)
    
    print(f"✅ Data loaded successfully!")
    print(f"Dataset shape: {spacex_df.shape}")
    
    # Filter to relevant columns
    spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
    
    # Group by Launch Site to get unique sites
    launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
    launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
    
    # Display statistics
    print(f"\n📊 Launch Statistics by Site:")
    print("-" * 50)
    
    for site in launch_sites_df['Launch Site']:
        site_data = spacex_df[spacex_df['Launch Site'] == site]
        total_launches = len(site_data)
        success_count = site_data['class'].sum()
        failure_count = total_launches - success_count
        
        print(f"{site}:")
        print(f"  Total: {total_launches}")
        print(f"  Success: {success_count} ({success_count/total_launches*100:.1f}%)")
        print(f"  Failed: {failure_count} ({failure_count/total_launches*100:.1f}%)")
    
    # Define NASA coordinates
    nasa_coordinate = [28.562302, -80.577356]
    
    # Initialize the map
    site_map = folium.Map(location=nasa_coordinate, zoom_start=5, tiles='OpenStreetMap')
    
    # Add title to the map
    title_html = '''
                 <h3 align="center" style="font-size:16px">
                 <b>SpaceX Launch Outcomes by Site</b><br>
                 Green = Success | Red = Failure
                 </h3>
                 '''
    site_map.get_root().html.add_child(folium.Element(title_html))
    
    # Create separate MarkerClusters for better organization
    success_cluster = MarkerCluster(name='Successful Launches').add_to(site_map)
    failure_cluster = MarkerCluster(name='Failed Launches').add_to(site_map)
    
    print(f"\n📍 Adding launch markers to map...")
    
    # For each launch in the dataset, add a marker based on success/failure
    for index, row in spacex_df.iterrows():
        launch_site = row['Launch Site']
        latitude = row['Lat']
        longitude = row['Long']
        launch_class = row['class']  # 1 = success, 0 = failure
        
        # Determine marker properties based on success/failure
        if launch_class == 1:  # Successful launch
            color = 'green'
            icon = 'check-circle'
            cluster = success_cluster
            outcome = 'Success'
        else:  # Failed launch
            color = 'red'
            icon = 'times-circle'
            cluster = failure_cluster
            outcome = 'Failure'
        
        # Create popup with launch information
        popup_content = f"""
        <div style="width: 200px;">
            <h4 style="margin: 5px 0; color: {'green' if outcome=='Success' else 'red'}">
                {outcome}
            </h4>
            <hr style="margin: 5px 0;">
            <p style="margin: 5px 0;"><b>Site:</b> {launch_site}</p>
            <p style="margin: 5px 0;"><b>Coordinates:</b></p>
            <p style="margin: 5px 0;">Lat: {latitude:.4f}</p>
            <p style="margin: 5px 0;">Long: {longitude:.4f}</p>
        </div>
        """
        
        # Add marker to the appropriate cluster
        folium.Marker(
            location=[latitude, longitude],
            popup=folium.Popup(popup_content, max_width=250),
            tooltip=f"{outcome} at {launch_site}",
            icon=folium.Icon(
                color=color,
                icon=icon,
                prefix='fa'
            )
        ).add_to(cluster)
    
    # Add circles for each launch site location
    for index, row in launch_sites_df.iterrows():
        folium.Circle(
            location=[row['Lat'], row['Long']],
            radius=2000,  # 2 km radius
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.1,
            popup=f"Launch Site: {row['Launch Site']}",
            tooltip=f"Launch Site: {row['Launch Site']}"
        ).add_to(site_map)
    
    # Add mouse position display
    from folium.plugins import MousePosition
    MousePosition().add_to(site_map)
    
    # Add layer control to toggle between success/failure markers
    folium.LayerControl(collapsed=False).add_to(site_map)
    
    # Add a legend
    legend_html = '''
    <div style="position: fixed; 
         bottom: 50px; left: 50px; width: 180px; height: 140px; 
         border:2px solid grey; z-index:9999; font-size:14px;
         background-color:white;
         padding: 10px;">
         <b>Legend</b><br>
         <i class="fa fa-check-circle" style="color:green"></i> Successful Launch<br>
         <i class="fa fa-times-circle" style="color:red"></i> Failed Launch<br>
         <i class="fa fa-circle" style="color:blue"></i> Launch Site Area<br>
         <hr style="margin: 5px 0;">
         <i>Click clusters to zoom</i><br>
         <i>Use layer control to filter</i>
    </div>
    '''
    site_map.get_root().html.add_child(folium.Element(legend_html))
    
    print(f"\n✅ Map created successfully!")
    print(f"Total launches: {len(spacex_df)}")
    print(f"Successful: {spacex_df['class'].sum()}")
    print(f"Failed: {len(spacex_df) - spacex_df['class'].sum()}")
    
    # Display the map
    display(site_map)
    
    # Save the map to an HTML file
    output_file = 'spacex_launch_outcomes.html'
    site_map.save(output_file)
    print(f"💾 Map saved to: {output_file}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n🔧 If you can't load the data, here's a sample visualization approach:")
    
    # Create sample data for demonstration
    sample_data = {
        'Launch Site': ['CCAFS SLC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'KSC LC-39A', 'VAFB SLC-4E'],
        'Lat': [28.561857, 28.561857, 28.572872, 28.572872, 34.632093],
        'Long': [-80.577366, -80.577366, -80.648981, -80.648981, -120.610829],
        'class': [1, 0, 1, 1, 0]  # 1=success, 0=failure
    }
    
    sample_df = pd.DataFrame(sample_data)
    
    # Create a simple map with the sample data
    sample_map = folium.Map(location=[28.562302, -80.577356], zoom_start=5)
    
    for index, row in sample_df.iterrows():
        color = 'green' if row['class'] == 1 else 'red'
        icon = 'check-circle' if row['class'] == 1 else 'times-circle'
        outcome = 'Success' if row['class'] == 1 else 'Failure'
        
        folium.Marker(
            location=[row['Lat'], row['Long']],
            popup=f"{outcome} at {row['Launch Site']}",
            icon=folium.Icon(color=color, icon=icon, prefix='fa')
        ).add_to(sample_map)
    
    print("\n📍 Sample map created with demonstration data:")
    display(sample_map)

✅ Data loaded successfully!
Dataset shape: (56, 13)

📊 Launch Statistics by Site:
--------------------------------------------------
CCAFS LC-40:
  Total: 26
  Success: 7 (26.9%)
  Failed: 19 (73.1%)
CCAFS SLC-40:
  Total: 7
  Success: 3 (42.9%)
  Failed: 4 (57.1%)
KSC LC-39A:
  Total: 13
  Success: 10 (76.9%)
  Failed: 3 (23.1%)
VAFB SLC-4E:
  Total: 10
  Success: 4 (40.0%)
  Failed: 6 (60.0%)

📍 Adding launch markers to map...

✅ Map created successfully!
Total launches: 56
Successful: 24
Failed: 32


💾 Map saved to: spacex_launch_outcomes.html


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [15]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [16]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [17]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
import folium
import pandas as pd

# First, let's create or load your data
# For demonstration, I'll create sample data
data = {
    'Launch Site': ['CCAFS SLC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E', 'KSC LC-39A'],
    'Lat': [28.561857, 28.561857, 28.572872, 34.632093, 28.572872],
    'Long': [-80.577366, -80.577366, -80.648981, -120.610829, -80.648981],
    'class': [1, 0, 1, 0, 1]  # 1 = success, 0 = failure
}

df = pd.DataFrame(data)
print("Sample DataFrame:")
print(df)
print()

# Define the function to check class value and return marker color
def get_marker_color(class_value):
    """
    Returns marker color based on class value.
    
    Args:
        class_value: Integer (0 or 1)
    
    Returns:
        String: 'green' for class=1, 'red' for class=0
    """
    if class_value == 1:
        return 'green'
    elif class_value == 0:
        return 'red'
    else:
        return 'gray'  # Default for unexpected values

# Apply the function to create a new column
df['marker_color'] = df['class'].apply(get_marker_color)

print("DataFrame with marker_color column:")
print(df)
print()

# Create a map
map_center = [df['Lat'].mean(), df['Long'].mean()]
site_map = folium.Map(location=map_center, zoom_start=5)

# Add markers using the marker_color column
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=row['marker_color'], icon='rocket', prefix='fa')
    ).add_to(site_map)

display(site_map)
print("✅ Map created with color-coded markers!")

Sample DataFrame:
    Launch Site        Lat        Long  class
0  CCAFS SLC-40  28.561857  -80.577366      1
1  CCAFS SLC-40  28.561857  -80.577366      0
2    KSC LC-39A  28.572872  -80.648981      1
3   VAFB SLC-4E  34.632093 -120.610829      0
4    KSC LC-39A  28.572872  -80.648981      1

DataFrame with marker_color column:
    Launch Site        Lat        Long  class marker_color
0  CCAFS SLC-40  28.561857  -80.577366      1        green
1  CCAFS SLC-40  28.561857  -80.577366      0          red
2    KSC LC-39A  28.572872  -80.648981      1        green
3   VAFB SLC-4E  34.632093 -120.610829      0          red
4    KSC LC-39A  28.572872  -80.648981      1        green



✅ Map created with color-coded markers!


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [18]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map
# First, ensure you have the marker_color column in your dataframe
# If not already created, add it:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Initialize the map with NASA coordinates
nasa_coordinate = [28.562302, -80.577356]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame, create a Marker object
for index, record in spacex_df.iterrows():
    # Determine icon properties based on class value
    if record['class'] == 1:  # Success
        icon_color = 'green'
        icon_type = 'check-circle'
        outcome = 'Success'
    else:  # Failure (class == 0)
        icon_color = 'red'
        icon_type = 'times-circle'
        outcome = 'Failure'
    
    # Create the Marker object
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=folium.Popup(
            f"""
            <div style="width: 200px;">
                <h4 style="color: {icon_color}">{outcome}</h4>
                <hr>
                <p><b>Launch Site:</b> {record['Launch Site']}</p>
                <p><b>Coordinates:</b></p>
                <p>Lat: {record['Lat']:.4f}</p>
                <p>Long: {record['Long']:.4f}</p>
                <p><b>Class Value:</b> {record['class']}</p>
            </div>
            """,
            max_width=250
        ),
        tooltip=f"{outcome} at {record['Launch Site']}",
        icon=folium.Icon(
            color='white',  # Background color of the icon
            icon_color=icon_color,  # Color of the icon itself
            icon=icon_type,  # Type of icon
            prefix='fa'  # Font Awesome icons
        )
    )
    
    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Add layer control and other enhancements
from folium.plugins import MousePosition

# Add mouse position
MousePosition().add_to(site_map)

# Add a legend
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 180px; height: 100px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;">
     <b>Launch Outcomes</b><br>
     <i class="fa fa-check-circle" style="color:green"></i> Success<br>
     <i class="fa fa-times-circle" style="color:red"></i> Failure<br>
     <hr style="margin: 5px 0;">
     <i>Click clusters to expand</i>
</div>
'''
site_map.get_root().html.add_child(folium.Element(legend_html))

# Display the map
display(site_map)

print(f"✅ Marker cluster added to map!")
print(f"Total markers: {len(spacex_df)}")
print(f"Successful launches: {spacex_df['class'].sum()}")
print(f"Failed launches: {len(spacex_df) - spacex_df['class'].sum()}")

✅ Marker cluster added to map!
Total markers: 56
Successful launches: 24
Failed launches: 32


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [19]:
# TASK 3: Calculate the distances between a launch site to its proximities
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# First, let's get our launch site data
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

print("📍 Launch Sites:")
print(launch_sites_df)
print()

# Define some major nearby cities/points for each launch site
# These are approximate coordinates - you can adjust them
nearby_locations = {
    'CCAFS SLC-40': [
        {'name': 'Cape Canaveral City', 'lat': 28.4058, 'lon': -80.6048},
        {'name': 'Port Canaveral', 'lat': 28.4103, 'lon': -80.6271},
        {'name': 'Kennedy Space Center', 'lat': 28.5729, 'lon': -80.6490},
        {'name': 'Titusville', 'lat': 28.6122, 'lon': -80.8076},
    ],
    'KSC LC-39A': [
        {'name': 'Kennedy Space Center VC', 'lat': 28.5241, 'lon': -80.6509},
        {'name': 'Titusville Downtown', 'lat': 28.6122, 'lon': -80.8076},
        {'name': 'Merritt Island', 'lat': 28.3578, 'lon': -80.6923},
        {'name': 'Cocoa Beach', 'lat': 28.3200, 'lon': -80.6076},
    ],
    'VAFB SLC-4E': [
        {'name': 'Lompoc City', 'lat': 34.6392, 'lon': -120.4579},
        {'name': 'Vandenberg Village', 'lat': 34.7083, 'lon': -120.4677},
        {'name': 'Santa Maria', 'lat': 34.9530, 'lon': -120.4357},
        {'name': 'Point Arguello', 'lat': 34.5772, 'lon': -120.6483},
    ]
}

# Haversine formula to calculate distance between two coordinates
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    
    Returns distance in kilometers
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    # Radius of earth in kilometers
    r = 6371.0
    
    return c * r

# Calculate distances for each launch site
print("📏 DISTANCES FROM LAUNCH SITES TO NEARBY LOCATIONS:")
print("=" * 80)

all_distances = []

for index, site_row in launch_sites_df.iterrows():
    site_name = site_row['Launch Site']
    site_lat = site_row['Lat']
    site_lon = site_row['Long']
    
    print(f"\n🚀 {site_name}:")
    print(f"   Coordinates: ({site_lat:.4f}, {site_lon:.4f})")
    
    if site_name in nearby_locations:
        for location in nearby_locations[site_name]:
            distance_km = haversine_distance(
                site_lat, site_lon,
                location['lat'], location['lon']
            )
            
            # Convert to miles (optional)
            distance_miles = distance_km * 0.621371
            
            print(f"   → {location['name']}:")
            print(f"       {distance_km:.2f} km ({distance_miles:.2f} miles)")
            
            # Store for analysis
            all_distances.append({
                'Launch Site': site_name,
                'Nearby Location': location['name'],
                'Distance_km': distance_km,
                'Distance_miles': distance_miles,
                'Location_Lat': location['lat'],
                'Location_Lon': location['lon']
            })
    else:
        print(f"   ⚠️  No nearby locations defined for this site")

# Create a DataFrame with all distances
distances_df = pd.DataFrame(all_distances)

print("\n" + "=" * 80)
print("📊 SUMMARY STATISTICS:")
print("=" * 80)

# Calculate statistics for each launch site
for site in distances_df['Launch Site'].unique():
    site_distances = distances_df[distances_df['Launch Site'] == site]
    
    print(f"\n📌 {site}:")
    print(f"   Closest: {site_distances.loc[site_distances['Distance_km'].idxmin(), 'Nearby Location']}")
    print(f"           {site_distances['Distance_km'].min():.2f} km")
    
    print(f"   Farthest: {site_distances.loc[site_distances['Distance_km'].idxmax(), 'Nearby Location']}")
    print(f"            {site_distances['Distance_km'].max():.2f} km")
    
    print(f"   Average: {site_distances['Distance_km'].mean():.2f} km")
    print(f"   Std Dev: {site_distances['Distance_km'].std():.2f} km")

# Find closest launch site to populated areas
print("\n" + "=" * 80)
print("🏙️  CLOSEST POPULATED AREAS TO LAUNCH SITES:")
print("=" * 80)

for site in distances_df['Launch Site'].unique():
    closest = distances_df[distances_df['Launch Site'] == site].nsmallest(1, 'Distance_km')
    print(f"{site}: {closest['Nearby Location'].values[0]} ({closest['Distance_km'].values[0]:.1f} km)")

📍 Launch Sites:
    Launch Site        Lat        Long
0   CCAFS LC-40  28.562302  -80.577356
1  CCAFS SLC-40  28.563197  -80.576820
2    KSC LC-39A  28.573255  -80.646895
3   VAFB SLC-4E  34.632834 -120.610745

📏 DISTANCES FROM LAUNCH SITES TO NEARBY LOCATIONS:

🚀 CCAFS LC-40:
   Coordinates: (28.5623, -80.5774)
   ⚠️  No nearby locations defined for this site

🚀 CCAFS SLC-40:
   Coordinates: (28.5632, -80.5768)
   → Cape Canaveral City:
       17.71 km (11.01 miles)
   → Port Canaveral:
       17.70 km (11.00 miles)
   → Kennedy Space Center:
       7.13 km (4.43 miles)
   → Titusville:
       23.18 km (14.40 miles)

🚀 KSC LC-39A:
   Coordinates: (28.5733, -80.6469)
   → Kennedy Space Center VC:
       5.48 km (3.40 miles)
   → Titusville Downtown:
       16.28 km (10.11 miles)
   → Merritt Island:
       24.37 km (15.14 miles)
   → Cocoa Beach:
       28.42 km (17.66 miles)

🚀 VAFB SLC-4E:
   Coordinates: (34.6328, -120.6107)
   → Lompoc City:
       14.00 km (8.70 miles)
   → Vande

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [20]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [21]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [23]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# Haversine distance function
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate distance between two coordinates in kilometers."""
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return 6371.0 * c

# Get launch sites data
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

print("📍 SpaceX Launch Sites:")
print(launch_sites_df)
print()

# Define coastline points near each launch site
# These are actual coastline coordinates near SpaceX launch sites
coastline_points = {
    'CCAFS SLC-40': [
        # Cape Canaveral coastline points
        {'name': 'Cape Canaveral Coast 1', 'lat': 28.4582, 'lon': -80.5284},
        {'name': 'Cape Canaveral Coast 2', 'lat': 28.4621, 'lon': -80.5548},
        {'name': 'Playalinda Beach', 'lat': 28.6467, 'lon': -80.6145},
        {'name': 'Cocoa Beach', 'lat': 28.3140, 'lon': -80.6077},
        {'name': 'Port Canaveral Entrance', 'lat': 28.4104, 'lon': -80.5889},
    ],
    'KSC LC-39A': [
        # Kennedy Space Center coastline
        {'name': 'KSC Coastline 1', 'lat': 28.5256, 'lon': -80.5754},
        {'name': 'KSC Coastline 2', 'lat': 28.5832, 'lon': -80.5687},
        {'name': 'Banana River Entrance', 'lat': 28.4074, 'lon': -80.6112},
        {'name': 'Mosquito Lagoon', 'lat': 28.7551, 'lon': -80.7430},
    ],
    'VAFB SLC-4E': [
        # Vandenberg coastline points
        {'name': 'Surf Beach', 'lat': 34.6821, 'lon': -120.6224},
        {'name': 'Ocean Beach Park', 'lat': 34.6736, 'lon': -120.6168},
        {'name': 'Point Arguello Coast', 'lat': 34.5772, 'lon': -120.6483},
        {'name': 'Jalama Beach', 'lat': 34.5106, 'lon': -120.5019},
    ]
}

print("🌊 FINDING CLOSEST COASTLINE FOR EACH LAUNCH SITE:")
print("=" * 80)

coastline_results = []

for index, site_row in launch_sites_df.iterrows():
    site_name = site_row['Launch Site']
    site_lat = site_row['Lat']
    site_lon = site_row['Long']
    
    print(f"\n🚀 {site_name}:")
    print(f"   Launch Site Coordinates: ({site_lat:.6f}, {site_lon:.6f})")
    
    if site_name in coastline_points:
        # Find closest coastline point
        min_distance = float('inf')
        closest_coast = None
        
        for coast_point in coastline_points[site_name]:
            distance = haversine_distance(
                site_lat, site_lon,
                coast_point['lat'], coast_point['lon']
            )
            
            print(f"   → {coast_point['name']}: {distance:.4f} km")
            
            if distance < min_distance:
                min_distance = distance
                closest_coast = coast_point
        
        # Convert to miles
        distance_miles = min_distance * 0.621371
        
        print(f"\n   ✅ CLOSEST COASTLINE:")
        print(f"       Name: {closest_coast['name']}")
        print(f"       Coordinates: Lat: {closest_coast['lat']:.5f}, Lon: {closest_coast['lon']:.5f}")
        print(f"       Distance: {min_distance:.4f} km ({distance_miles:.4f} miles)")
        
        # Store results
        coastline_results.append({
            'Launch Site': site_name,
            'Site_Lat': site_lat,
            'Site_Lon': site_lon,
            'Coastline_Name': closest_coast['name'],
            'Coastline_Lat': closest_coast['lat'],
            'Coastline_Lon': closest_coast['lon'],
            'Distance_km': min_distance,
            'Distance_miles': distance_miles
        })
    else:
        print(f"   ⚠️  No coastline points defined for this site")

# Create results DataFrame
coastline_df = pd.DataFrame(coastline_results)

print("\n" + "=" * 80)
print("📊 COASTLINE DISTANCE SUMMARY:")
print("=" * 80)

print("\n" + coastline_df.to_string(index=False))

# Summary statistics
print(f"\n📈 STATISTICS:")
print(f"   Average distance to coastline: {coastline_df['Distance_km'].mean():.3f} km")
print(f"   Closest to coastline: {coastline_df.loc[coastline_df['Distance_km'].idxmin(), 'Launch Site']} "
      f"({coastline_df['Distance_km'].min():.3f} km)")
print(f"   Farthest from coastline: {coastline_df.loc[coastline_df['Distance_km'].idxmax(), 'Launch Site']} "
      f"({coastline_df['Distance_km'].max():.3f} km)")

📍 SpaceX Launch Sites:
    Launch Site        Lat        Long
0   CCAFS LC-40  28.562302  -80.577356
1  CCAFS SLC-40  28.563197  -80.576820
2    KSC LC-39A  28.573255  -80.646895
3   VAFB SLC-4E  34.632834 -120.610745

🌊 FINDING CLOSEST COASTLINE FOR EACH LAUNCH SITE:

🚀 CCAFS LC-40:
   Launch Site Coordinates: (28.562302, -80.577356)
   ⚠️  No coastline points defined for this site

🚀 CCAFS SLC-40:
   Launch Site Coordinates: (28.563197, -80.576820)
   → Cape Canaveral Coast 1: 12.5973 km
   → Cape Canaveral Coast 2: 11.4455 km
   → Playalinda Beach: 9.9872 km
   → Cocoa Beach: 27.8735 km
   → Port Canaveral Entrance: 17.0312 km

   ✅ CLOSEST COASTLINE:
       Name: Playalinda Beach
       Coordinates: Lat: 28.64670, Lon: -80.61450
       Distance: 9.9872 km (6.2057 miles)

🚀 KSC LC-39A:
   Launch Site Coordinates: (28.573255, -80.646895)
   → KSC Coastline 1: 8.7661 km
   → KSC Coastline 2: 7.7152 km
   → Banana River Entrance: 18.7692 km
   → Mosquito Lagoon: 22.2886 km

   ✅ CLOSES

In [24]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

import folium
from folium.features import DivIcon
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

# Haversine distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate distance between two coordinates in kilometers."""
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return 6371.0 * c

# Get launch sites data
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Define specific coastline points (using your example)
coastline_points = {
    'CCAFS SLC-40': {'lat': 28.4582, 'lon': -80.5284, 'name': 'Cape Canaveral Coast'},
    'KSC LC-39A': {'lat': 28.56367, 'lon': -80.57163, 'name': 'KSC Beach'},  # Your specified coordinate
    'VAFB SLC-4E': {'lat': 34.6821, 'lon': -120.6224, 'name': 'Surf Beach'}
}

print("📍 CALCULATING COASTLINE DISTANCES:")
print("=" * 80)

# Calculate distances and prepare data
coastline_data = []

for index, site_row in launch_sites_df.iterrows():
    site_name = site_row['Launch Site']
    site_lat = site_row['Lat']
    site_lon = site_row['Long']
    
    if site_name in coastline_points:
        coast_point = coastline_points[site_name]
        distance = calculate_distance(
            site_lat, site_lon,
            coast_point['lat'], coast_point['lon']
        )
        
        print(f"\n🚀 {site_name}:")
        print(f"   Launch Site: ({site_lat:.6f}, {site_lon:.6f})")
        print(f"   Coastline: {coast_point['name']}")
        print(f"   Coordinates: ({coast_point['lat']:.6f}, {coast_point['lon']:.6f})")
        print(f"   Distance: {distance:.4f} km ({distance * 0.621371:.4f} miles)")
        
        coastline_data.append({
            'Launch Site': site_name,
            'Site_Lat': site_lat,
            'Site_Lon': site_lon,
            'Coast_Name': coast_point['name'],
            'Coast_Lat': coast_point['lat'],
            'Coast_Lon': coast_point['lon'],
            'Distance_km': distance,
            'Distance_miles': distance * 0.621371
        })

# Create a map
center_lat = launch_sites_df['Lat'].mean()
center_lon = launch_sites_df['Long'].mean()

coastline_map = folium.Map(
    location=[center_lat, center_lon], 
    zoom_start=5,
    tiles='OpenStreetMap'
)

print("\n" + "=" * 80)
print("🗺️ CREATING MAP WITH COASTLINE MARKERS AND DISTANCE LABELS")
print("=" * 80)

# Add launch site markers
for site_data in coastline_data:
    # Launch site marker
    folium.Marker(
        location=[site_data['Site_Lat'], site_data['Site_Lon']],
        popup=f"🚀 {site_data['Launch Site']}",
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(coastline_map)

# Add coastline markers with distance labels
for site_data in coastline_data:
    # Create the distance marker on the coastline point
    distance_marker = folium.Marker(
        location=[site_data['Coast_Lat'], site_data['Coast_Lon']],
        icon=DivIcon(
            icon_size=(150, 36),  # Size of the label
            icon_anchor=(0, 0),   # Anchor point (top-left corner)
            html=f'''
            <div style="font-size: 12px; 
                 color: #d35400; 
                 font-weight: bold;
                 background-color: white;
                 padding: 5px;
                 border-radius: 4px;
                 border: 2px solid #d35400;
                 text-align: center;
                 min-width: 100px;">
                <b>{site_data["Distance_km"]:10.2f} KM</b><br>
                <span style="font-size: 10px; color: #7f8c8d;">
                to {site_data["Launch Site"]}
                </span>
            </div>
            '''
        )
    )
    
    # Add the marker to the map
    distance_marker.add_to(coastline_map)
    
    print(f"\n✅ Added distance marker for {site_data['Launch Site']}:")
    print(f"   Coastline: {site_data['Coast_Name']}")
    print(f"   Distance displayed: {site_data['Distance_km']:.2f} KM")
    
    # Also add a regular marker for the coastline point
    folium.Marker(
        location=[site_data['Coast_Lat'], site_data['Coast_Lon']],
        popup=f"🏖️ {site_data['Coast_Name']}<br>"
              f"Distance to {site_data['Launch Site']}: {site_data['Distance_km']:.2f} km",
        icon=folium.Icon(color='blue', icon='water', prefix='fa')
    ).add_to(coastline_map)
    
    # Draw a line connecting launch site to coastline
    points = [
        [site_data['Site_Lat'], site_data['Site_Lon']],
        [site_data['Coast_Lat'], site_data['Coast_Lon']]
    ]
    
    folium.PolyLine(
        points,
        color='green',
        weight=2,
        opacity=0.6,
        popup=f"Distance: {site_data['Distance_km']:.2f} km",
        tooltip=f"{site_data['Launch Site']} → {site_data['Coast_Name']}"
    ).add_to(coastline_map)

# Add a title
title_html = '''
<div style="position: fixed; 
     top: 10px; left: 50px; z-index: 1000; 
     background-color: white; padding: 10px; border-radius: 5px; 
     border: 2px solid #2c3e50;">
    <h3 style="margin: 0; color: #2c3e50;">🚀 SpaceX Launch Sites to Coastline Distances</h3>
    <p style="margin: 5px 0 0 0; font-size: 12px; color: #7f8c8d;">
        Orange labels show distance from each launch site to nearest coastline
    </p>
</div>
'''
coastline_map.get_root().html.add_child(folium.Element(title_html))

# Add a legend
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 200px; height: 160px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;">
     <b>Map Legend</b><br>
     <i class="fa fa-rocket" style="color:red"></i> Launch Site<br>
     <i class="fa fa-water" style="color:blue"></i> Coastline Point<br>
     <div style="background-color: #d35400; color: white; 
          padding: 3px; margin: 5px 0; border-radius: 3px;
          text-align: center; font-weight: bold;">
          📏 12.34 KM
     </div>
     Distance Label<br>
     <div style="border-bottom: 2px solid green; margin: 5px 0;"></div>
     Connection Line<br>
     <hr style="margin: 5px 0;">
     <i>Click markers for details</i>
</div>
'''
coastline_map.get_root().html.add_child(folium.Element(legend_html))

# Display the map
display(coastline_map)
print("\n✅ Map created with distance markers on coastline points!")

# Save the map
output_file = 'spacex_coastline_distances.html'
coastline_map.save(output_file)
print(f"💾 Map saved to: {output_file}")

📍 CALCULATING COASTLINE DISTANCES:

🚀 CCAFS SLC-40:
   Launch Site: (28.563197, -80.576820)
   Coastline: Cape Canaveral Coast
   Coordinates: (28.458200, -80.528400)
   Distance: 12.5973 km (7.8276 miles)

🚀 KSC LC-39A:
   Launch Site: (28.573255, -80.646895)
   Coastline: KSC Beach
   Coordinates: (28.563670, -80.571630)
   Distance: 7.4270 km (4.6149 miles)

🚀 VAFB SLC-4E:
   Launch Site: (34.632834, -120.610745)
   Coastline: Surf Beach
   Coordinates: (34.682100, -120.622400)
   Distance: 5.5809 km (3.4678 miles)

🗺️ CREATING MAP WITH COASTLINE MARKERS AND DISTANCE LABELS

✅ Added distance marker for CCAFS SLC-40:
   Coastline: Cape Canaveral Coast
   Distance displayed: 12.60 KM

✅ Added distance marker for KSC LC-39A:
   Coastline: KSC Beach
   Distance displayed: 7.43 KM

✅ Added distance marker for VAFB SLC-4E:
   Coastline: Surf Beach
   Distance displayed: 5.58 KM



✅ Map created with distance markers on coastline points!
💾 Map saved to: spacex_coastline_distances.html


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [34]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
# Create map with launch site to coastline connections
center_lat = launch_sites_df['Lat'].mean()
center_lon = launch_sites_df['Long'].mean()

# Create the main site map
site_map = folium.Map(
    location=[center_lat, center_lon], 
    zoom_start=5,
    tiles='Stamen Terrain'
)

print("\n" + "=" * 80)
print("🗺️ CREATING LAUNCH SITE TO COASTLINE CONNECTIONS")
print("=" * 80)

# Add markers for all launch sites
print("📍 Adding launch site markers...")
for index, site_row in launch_sites_df.iterrows():
    site_name = site_row['Launch Site']
    site_coords = [site_row['Lat'], site_row['Long']]
    
    folium.Marker(
        location=site_coords,
        popup=site_name,
        icon=folium.Icon(color='red', icon='rocket')
    ).add_to(site_map)
    print(f"  ✓ Added marker for {site_name}")

# Add PolyLines connecting launch sites to their coastlines
print("\n🏖️ Adding coastline connections...")
for index, site_row in launch_sites_df.iterrows():
    site_name = site_row['Launch Site']
    
    if site_name in coastline_data:
        site_coords = [site_row['Lat'], site_row['Long']]
        coast_coords = [coastline_data[site_name]['coast_lat'], 
                       coastline_data[site_name]['coast_lon']]
        
        # Calculate distance
        distance = calculate_distance(
            site_coords[0], site_coords[1],
            coast_coords[0], coast_coords[1]
        )
        
        # Create PolyLine for this connection
        folium.PolyLine(
            locations=[site_coords, coast_coords],
            weight=2,
            color='#3498db',
            opacity=0.7,
            popup=folium.Popup(
                f"""
                <div style="width: 220px;">
                    <h4 style="color: #3498db;">{site_name} to Coastline</h4>
                    <div style="background-color: #f8f9fa; padding: 10px; border-radius: 5px;">
                        <p><b>📏 Distance:</b> {distance:.2f} km</p>
                        <p><b>📍 Launch Site:</b><br>{site_coords[0]:.5f}, {site_coords[1]:.5f}</p>
                        <p><b>🏖️ Coastline:</b><br>{coast_coords[0]:.5f}, {coast_coords[1]:.5f}</p>
                    </div>
                </div>
                """,
                max_width=250
            ),
            tooltip=f"Click for details: {site_name} to coastline"
        ).add_to(site_map)
        
        print(f"  ✓ Added connection for {site_name} ({distance:.2f} km)")

print("\n✅ Map created successfully!")
print(f"   Total connections: {len([s for s in launch_sites_df['Launch Site'] if s in coastline_data])}")

# Display the map
display(site_map)


<class 'NameError'>: name 'lines' is not defined

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [35]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Create map for nearby features
center_lat = launch_sites_df['Lat'].mean()
center_lon = launch_sites_df['Long'].mean()

features_map = folium.Map(
    location=[center_lat, center_lon], 
    zoom_start=5,
    tiles='CartoDB positron'
)

print("\n" + "=" * 80)
print("🗺️ CREATING NEARBY FEATURES CONNECTIONS")
print("=" * 80)

# Sample nearby features data - replace with your actual data
# This is example data structure - you should replace it with your actual data
nearby_features = {
    'CCAFS LC-40': {
        'city': {'name': 'Cocoa Beach', 'lat': 28.3200, 'lon': -80.6076, 'distance_km': 15.2},
        'highway': {'name': 'Florida State Road 401', 'lat': 28.4100, 'lon': -80.6200, 'distance_km': 5.3},
        'railway': {'name': 'Florida East Coast Railway', 'lat': 28.4300, 'lon': -80.6100, 'distance_km': 8.7}
    },
    'CCAFS SLC-40': {
        'city': {'name': 'Cape Canaveral', 'lat': 28.3883, 'lon': -80.6036, 'distance_km': 12.5},
        'highway': {'name': 'SR 528', 'lat': 28.4150, 'lon': -80.6250, 'distance_km': 4.1}
    },
    'KSC LC-39A': {
        'city': {'name': 'Titusville', 'lat': 28.6122, 'lon': -80.8078, 'distance_km': 22.8},
        'railway': {'name': 'NASA Railroad', 'lat': 28.5850, 'lon': -80.6500, 'distance_km': 10.5}
    },
    'VAFB SLC-4E': {
        'city': {'name': 'Lompoc', 'lat': 34.6391, 'lon': -120.4579, 'distance_km': 18.3},
        'highway': {'name': 'US-101', 'lat': 34.6800, 'lon': -120.4700, 'distance_km': 12.1}
    }
}

# Color codes for different feature types
feature_colors = {
    'city': '#e74c3c',      # Red
    'highway': '#f39c12',   # Orange
    'railway': '#27ae60',   # Green
    'coastline': '#3498db', # Blue
    'airport': '#9b59b6'    # Purple
}

# Feature icons
feature_icons = {
    'city': 'building',
    'highway': 'road',
    'railway': 'train',
    'coastline': 'water',
    'airport': 'plane'
}

# Add launch site markers first
print("🚀 Adding launch site markers...")
for index, site_row in launch_sites_df.iterrows():
    site_name = site_row['Launch Site']
    site_coords = [site_row['Lat'], site_row['Long']]
    
    folium.Marker(
        location=site_coords,
        popup=f"""
        <div style="width: 200px;">
            <h4 style="color: #2c3e50;">🚀 {site_name}</h4>
            <div style="background-color: #f8f9fa; padding: 10px; border-radius: 5px;">
                <p><b>Latitude:</b> {site_coords[0]:.5f}</p>
                <p><b>Longitude:</b> {site_coords[1]:.5f}</p>
            </div>
        </div>
        """,
        icon=folium.Icon(color='black', icon='rocket', prefix='fa')
    ).add_to(features_map)
    
    print(f"  ✓ Added launch site: {site_name}")

# Add nearby features and draw lines
print("\n📍 Adding nearby features and connections...")
for index, site_row in launch_sites_df.iterrows():
    site_name = site_row['Launch Site']
    site_coords = [site_row['Lat'], site_row['Long']]
    
    if site_name in nearby_features:
        site_features = nearby_features[site_name]
        
        for feature_type, feature_data in site_features.items():
            feature_coords = [feature_data['lat'], feature_data['lon']]
            distance = feature_data['distance_km']
            feature_name = feature_data['name']
            
            # Add feature marker
            folium.Marker(
                location=feature_coords,
                popup=f"""
                <div style="width: 220px;">
                    <h4 style="color: {feature_colors.get(feature_type, '#2c3e50')};">{feature_name}</h4>
                    <div style="background-color: #f8f9fa; padding: 10px; border-radius: 5px;">
                        <p><b>Type:</b> {feature_type.title()}</p>
                        <p><b>Distance to {site_name}:</b> {distance:.1f} km</p>
                        <p><b>Coordinates:</b><br>{feature_coords[0]:.5f}, {feature_coords[1]:.5f}</p>
                    </div>
                </div>
                """,
                icon=folium.Icon(
                    color=feature_colors.get(feature_type, 'gray'),
                    icon=feature_icons.get(feature_type, 'info-sign'),
                    prefix='fa'
                )
            ).add_to(features_map)
            
            # Draw line from launch site to feature
            folium.PolyLine(
                locations=[site_coords, feature_coords],
                weight=1.5,
                color=feature_colors.get(feature_type, '#95a5a6'),
                opacity=0.6,
                dash_array='5, 5',
                popup=f"""
                <div style="width: 200px;">
                    <h5>Connection Details</h5>
                    <p><b>From:</b> {site_name}</p>
                    <p><b>To:</b> {feature_name}</p>
                    <p><b>Distance:</b> {distance:.1f} km</p>
                    <p><b>Type:</b> {feature_type}</p>
                </div>
                """
            ).add_to(features_map)
            
            print(f"  ✓ Added {feature_type}: {feature_name} ({distance:.1f} km from {site_name})")

print("\n✅ Nearby features map created successfully!")

# Add legend
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 180px; height: 200px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px; border-radius: 5px;">
     
     <h4 style="margin-top:0; color: #2c3e50;">Legend</h4>
     <p><i class="fa fa-rocket" style="color:black"></i> Launch Site</p>
     <p><i class="fa fa-building" style="color:#e74c3c"></i> City</p>
     <p><i class="fa fa-road" style="color:#f39c12"></i> Highway</p>
     <p><i class="fa fa-train" style="color:#27ae60"></i> Railway</p>
     <p><i class="fa fa-water" style="color:#3498db"></i> Coastline</p>
     <p style="margin-bottom: 5px;"><i class="fa fa-plane" style="color:#9b59b6"></i> Airport</p>
     <hr>
     <p>Lines show connections<br>to nearest features</p>
</div>
'''

features_map.get_root().html.add_child(folium.Element(legend_html))

# Display the map
display(features_map)


🗺️ CREATING NEARBY FEATURES CONNECTIONS
🚀 Adding launch site markers...
  ✓ Added launch site: CCAFS LC-40
  ✓ Added launch site: CCAFS SLC-40
  ✓ Added launch site: KSC LC-39A
  ✓ Added launch site: VAFB SLC-4E

📍 Adding nearby features and connections...


<ipython-input-35-ca4b0b9005be>:106: UserWarning: color argument of Icon should be one of: {'darkblue', 'white', 'orange', 'blue', 'pink', 'green', 'lightred', 'lightblue', 'red', 'lightgreen', 'cadetblue', 'black', 'darkgreen', 'darkpurple', 'gray', 'darkred', 'purple', 'lightgray', 'beige'}.
  icon=folium.Icon(


  ✓ Added city: Cocoa Beach (15.2 km from CCAFS LC-40)
  ✓ Added highway: Florida State Road 401 (5.3 km from CCAFS LC-40)
  ✓ Added railway: Florida East Coast Railway (8.7 km from CCAFS LC-40)
  ✓ Added city: Cape Canaveral (12.5 km from CCAFS SLC-40)
  ✓ Added highway: SR 528 (4.1 km from CCAFS SLC-40)
  ✓ Added city: Titusville (22.8 km from KSC LC-39A)
  ✓ Added railway: NASA Railroad (10.5 km from KSC LC-39A)
  ✓ Added city: Lompoc (18.3 km from VAFB SLC-4E)
  ✓ Added highway: US-101 (12.1 km from VAFB SLC-4E)

✅ Nearby features map created successfully!


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
